# Benchmark NN

In [3]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import roc_curve, auc, accuracy_score
from sklearn.datasets import fetch_openml
import tensorflow as tf
from keras.layers import *
from qkeras import *
from qkeras.utils import load_qmodel
import hls4ml

## Training data

In [4]:
data = fetch_openml('hls4ml_lhc_jets_hlf')
X, Y = data['data'].to_numpy(), data['target'].to_numpy()

print(data['feature_names'])
print(X.shape, Y.shape)
print(Y[:12])

LE = LabelEncoder()
Y = LE.fit_transform(Y)
print(Y[:12])

Y = to_categorical(Y, 5)
print(Y[:12])

train_ratio = 0.6
val_ratio = 0.2
test_ratio = 1 - train_ratio - val_ratio

X_train_val, X_test, Y_train_val, Y_test = train_test_split(X, Y, test_size = test_ratio, random_state=15)
X_train, X_val, Y_train, Y_val = train_test_split(X_train_val, Y_train_val, test_size = val_ratio/(val_ratio+train_ratio), random_state=15)

scaler = StandardScaler().fit(X_train_val)
del X_train_val
X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

print("\nPartition the data set into train/val/test: %.2f/%.2f/%.2f" % (train_ratio, val_ratio, test_ratio))
print("X_train.shape: " + str(X_train.shape))
print("Y_train.shape: " + str(Y_train.shape))
print("X_val.shape: " + str(X_val.shape))
print("Y_val.shape: " + str(Y_val.shape))
print("X_test.shape: " + str(X_test.shape))
print("Y_test.shape: " + str(Y_test.shape))

['zlogz', 'c1_b0_mmdt', 'c1_b1_mmdt', 'c1_b2_mmdt', 'c2_b1_mmdt', 'c2_b2_mmdt', 'd2_b1_mmdt', 'd2_b2_mmdt', 'd2_a1_b1_mmdt', 'd2_a1_b2_mmdt', 'm2_b1_mmdt', 'm2_b2_mmdt', 'n2_b1_mmdt', 'n2_b2_mmdt', 'mass_mmdt', 'multiplicity']
(830000, 16) (830000,)
['g' 'w' 't' 'z' 'w' 'w' 't' 'g' 'z' 'g' 't' 'q']
[0 3 2 4 3 3 2 0 4 0 2 1]
[[1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0.]]

Partition the data set into train/val/test: 0.60/0.20/0.20
X_train.shape: (498000, 16)
Y_train.shape: (498000, 5)
X_val.shape: (166000, 16)
Y_val.shape: (166000, 5)
X_test.shape: (166000, 16)
Y_test.shape: (166000, 5)


## Set all possible precisions

In [5]:
# to match qkeras precision to ap_fixed precision
# for ap_fixed<B,I>, where one bit in I is for sign
# corresponding to qkeras' quantized_bits(B-1,I-1,symmetric=1,alpha=1) for weight/bias
# and quantized_relu(B-1,I-1)

# ap_fixed<B,I>
I=[4,5,6,7,8,9,10]
B=[]
for j in range(len(I)):
    B.append([])
    # at least 2 fractional bits, and up to 18 total bits
    for k in range(I[j]+2,18+1):
        B[j].append(k)

# qkeras precisions with total bit = b, integer bit = i
i=[]
b=[]
for j in range(len(I)):
    i.append(I[j]-1)
    b.append([])
    for k in range(len(B[j])):
        b[j].append(B[j][k]-1)
        
print('Loop over precisions, for each ap_fixed<B,I> we want and corresponding qkeras<b,i> for training:')
for j in range(len(I)):
    print('============================================================')
    print('ap_fixed<B,I>')
    print('I={}'.format(I[j]))
    print('B={}'.format(B[j]))
    print('------------------------------------------------------------')
    print('qkeras<b,i>')
    print('i={}'.format(i[j]))
    print('b={}'.format(b[j]))

assert(len(B)==len(b))
assert(len(I)==len(i))

Loop over precisions, for each ap_fixed<B,I> we want and corresponding qkeras<b,i> for training:
ap_fixed<B,I>
I=4
B=[6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
------------------------------------------------------------
qkeras<b,i>
i=3
b=[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]
ap_fixed<B,I>
I=5
B=[7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
------------------------------------------------------------
qkeras<b,i>
i=4
b=[6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]
ap_fixed<B,I>
I=6
B=[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
------------------------------------------------------------
qkeras<b,i>
i=5
b=[7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]
ap_fixed<B,I>
I=7
B=[9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
------------------------------------------------------------
qkeras<b,i>
i=6
b=[8, 9, 10, 11, 12, 13, 14, 15, 16, 17]
ap_fixed<B,I>
I=8
B=[10, 11, 12, 13, 14, 15, 16, 17, 18]
------------------------------------------------------------
qkeras<b,i>
i=7
b=[9, 10, 11, 12, 13,

## Compile and train qkeras models, looping over all precisions

In [ ]:
# note again that ap_fixed<B,I> corresponds to qkeras<b=B-1,i=I-1>
for j in range(len(i)):
    for k in range(len(b[j])):
        print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
        print('creating model for ap_fixed<{},{}>, or qkeras<{},{}>...'.format(B[j][k],I[j],b[j][k],i[j]))
        x_in = Input(16, name='in')
        x = QDense(64,
                  kernel_quantizer=quantized_bits(b[j][k],i[j],symmetric=1,alpha=1),
                  bias_quantizer=quantized_bits(b[j][k],i[j],symmetric=1,alpha=1), name="qdense_1")(x_in)
        x = QActivation('quantized_relu({},{})'.format(b[j][k],i[j]), name='qrelu_1')(x)
        x = QDense(32,
                  kernel_quantizer=quantized_bits(b[j][k],i[j],symmetric=1,alpha=1),
                  bias_quantizer=quantized_bits(b[j][k],i[j],symmetric=1,alpha=1), name="qdense_2")(x)
        x = QActivation('quantized_relu({},{})'.format(b[j][k],i[j]), name='qrelu_2')(x)
        x = QDense(32,
                  kernel_quantizer=quantized_bits(b[j][k],i[j],symmetric=1,alpha=1),
                  bias_quantizer=quantized_bits(b[j][k],i[j],symmetric=1,alpha=1), name="qdense_3")(x)
        x = QActivation('quantized_relu({},{})'.format(b[j][k],i[j]), name='qrelu_3')(x)
        x = QDense(5,
                  kernel_quantizer=quantized_bits(b[j][k],i[j],symmetric=1,alpha=1),
                  bias_quantizer=quantized_bits(b[j][k],i[j],symmetric=1,alpha=1), name="qdense_4")(x)
        x = Activation('softmax', name='softmax')(x)

        model = tf.keras.models.Model(x_in, x)
        #model.summary()
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')
        
        history = model.fit(X_train, Y_train,
                    epochs = 30,
                    validation_data = (X_val, Y_val),
                    batch_size = 256)
        
        plt.figure(figsize = (15,10))
        axes = plt.subplot(2, 2, 1)
        axes.plot(history.history['loss'], label = 'train loss')
        axes.plot(history.history['val_loss'], label = 'val loss')
        axes.legend(loc = "upper right")
        axes.set_xlabel('epoch')
        axes.set_ylabel('loss')
        plt.show()
        
        Y = model.predict(X_test)
        print("Accuracy: {}".format(accuracy_score(np.argmax(Y_test, axis=1), np.argmax(Y, axis=1))))

        model.save('model_nn_qat/qkeras_model_ap_fixed_{}-{}'.format(B[j][k],I[j]))
        print('done and saved to model_nn_qat/qkeras_model_ap_fixed_{}-{}'.format(B[j][k],I[j]))

## Convert to hls models, looping over all precisions

In [ ]:
tf.get_logger().setLevel('ERROR')
for j in range(len(I)):
    for k in range(len(B[j])):
        print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
        model = load_qmodel('model_nn_qat/qkeras_model_ap_fixed_{}-{}'.format(B[j][k],I[j]))
        config = hls4ml.utils.config_from_keras_model(model, granularity='model')
        config['Model']['Precision'] = {'accum': 'ap_fixed<18,8>', 'default': 'ap_fixed<{},{}>'.format(B[j][k],I[j])}
        hls_model = hls4ml.converters.convert_from_keras_model(model,
                                                       hls_config=config,
                                                       output_dir='model_nn_qat/hls_model_ap_fixed_{}-{}'.format(B[j][k],I[j]))
        hls_model.write()
        hls_model.compile()
        
        Y = model.predict(X_test)
        Y_hls = hls_model.predict(X_test)
        
        print('qkeras accuracy: {}'.format(accuracy_score(np.argmax(Y_test, axis=1), np.argmax(Y, axis=1))))
        print('hls accuracy: {}'.format(accuracy_score(np.argmax(Y_test, axis=1), np.argmax(Y_hls, axis=1))))
        print('hls model saved to model_nn_qat/hls_model_ap_fixed_{}-{}'.format(B[j][k],I[j]))